In [1]:
#imports
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks

Using TensorFlow backend.


In [2]:
DEV = False
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
  DEV = True

if DEV:
  epochs = 12
else:
  epochs = 50

train_data_path = './data/train'
validation_data_path = './data/validation'

"""
Parameters
"""
img_width, img_height = 150, 150
batch_size = 64
samples_per_epoch = 4000
validation_steps = 1000
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 8
lr = 0.0004

In [6]:
model = Sequential()
model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Conv2D(nb_filters2, (conv2_size, conv2_size), padding ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format='channels_first'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 6631 images belonging to 8 classes.
Found 6631 images belonging to 8 classes.


In [11]:
"""
Tensorboard log
"""
log_dir = './tf-log/'
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)
cbks = [tb_cb]

#model.fit_generator(
#    train_generator,
#    samples_per_epoch=samples_per_epoch,
#    epochs=epochs,
#    validation_data=validation_generator,
#    callbacks=cbks,
#    validation_steps=validation_steps)

model.fit_generator(
    train_generator,
    steps_per_epoch=samples_per_epoch // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps // batch_size)

target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./models/modelv2.0_8.categories.h5')
model.save_weights('./models/weights_v2.0_8.categories.0.h5')

Epoch 1/50
62/62 [==============================] - 464s 7s/step - loss: 2.5334 - acc: 0.3079 - val_loss: 1.6095 - val_acc: 0.4646
Epoch 2/50
62/62 [==============================] - 424s 7s/step - loss: 1.6494 - acc: 0.4057 - val_loss: 1.4419 - val_acc: 0.4750
Epoch 3/50
62/62 [==============================] - 402s 6s/step - loss: 1.5417 - acc: 0.4484 - val_loss: 1.3450 - val_acc: 0.5146
Epoch 4/50
62/62 [==============================] - 390s 6s/step - loss: 1.5117 - acc: 0.4698 - val_loss: 1.3085 - val_acc: 0.5417
Epoch 5/50
62/62 [==============================] - 394s 6s/step - loss: 1.4655 - acc: 0.4888 - val_loss: 1.2377 - val_acc: 0.5719
Epoch 6/50
62/62 [==============================] - 399s 6s/step - loss: 1.4079 - acc: 0.5204 - val_loss: 1.2395 - val_acc: 0.5865
Epoch 7/50
62/62 [==============================] - 394s 6s/step - loss: 1.3809 - acc: 0.5270 - val_loss: 1.2101 - val_acc: 0.5594
Epoch 8/50
62/62 [==============================] - 402s 6s/step - loss: 1.3687 - a

In [19]:
#predict_multiclass
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model



In [33]:
img_width, img_height = 150, 150
model_path = './models/modelv2.0_8.categories.h5'
model_weights_path = './models/weights_v2.0_8.categories.0.h5'
model = load_model(model_path)
model.load_weights(model_weights_path)

def predict(file):
  x = load_img(file, target_size=(img_width,img_height))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = model.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("Label: banana")
  elif answer == 1:
    print("Labels: battery")
  elif answer == 2:
    print("Label: computer")
  elif answer == 3:
    print("Label: glass_bottel")
  elif answer == 4:
    print("Label: light_blub")
  elif answer == 5:
    print("Label: paper")
  elif answer == 6:
    print("Label: phone")
  elif answer == 7:
    print("Label: plastic")
  

  return answer

In [45]:
# classify a picture
for i, ret in enumerate(os.walk('./test-data/whatishere')):
  for i, filename in enumerate(ret[2]):
    result = predict(ret[0] + '/' + filename)

Label: phone


In [37]:
banana_t = 0
banana_f = 0

battery_t = 0
battery_f = 0

computer_t = 0
computer_f = 0

glass_bottle_t = 0
glass_bottle_f = 0

light_blub_t = 0
light_blub_f = 0

paper_t = 0
paper_f = 0

phone_t = 0
phone_f = 0

plastic_t = 0
plastic_f = 0

#banana
for i, ret in enumerate(os.walk('./test-data/banana')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: Banana")
    result = predict(ret[0] + '/' + filename)
    if result == 0:
      banana_t += 1
    else:
      banana_f += 1

'''
#battery
for i, ret in enumerate(os.walk('./test-data/battery')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: Battery")
    result = predict(ret[0] + '/' + filename)
    if result == 1:
      battery_t += 1
    else:
      battery_f += 1

#computer
for i, ret in enumerate(os.walk('./test-data/computer')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: Computer")
    result = predict(ret[0] + '/' + filename)
    if result == 2:
      computer_t += 1
    else:
      computer_f += 1
    
#glass_bottle
for i, ret in enumerate(os.walk('./test-data/glass_bottle')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: glass bottle")
    result = predict(ret[0] + '/' + filename)
    if result == 0:
      glass_bottle_t += 1
    else:
      glass_bottle_f += 1

#light_blub
for i, ret in enumerate(os.walk('./test-data/light_blub')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: light blub")
    result = predict(ret[0] + '/' + filename)
    if result == 0:
      light_blub_t += 1
    else:
      light_blub_f += 1

#paper
for i, ret in enumerate(os.walk('./test-data/paper')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: Paper")
    result = predict(ret[0] + '/' + filename)
    if result == 0:
      paper_t += 1
    else:
      paper_f += 1

#phone
for i, ret in enumerate(os.walk('./test-data/phone')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: Phone")
    result = predict(ret[0] + '/' + filename)
    if result == 0:
      phone_t += 1
    else:
      phone_f += 1

#plastic
for i, ret in enumerate(os.walk('./test-data/plastic')):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    print("Label: Plastic")
    result = predict(ret[0] + '/' + filename)
    if result == 0:
      plastic_t += 1
    else:
      plastic_f += 1
      
'''

"""
Check metrics
"""
print("True Banana: ", banana_t)
print("False Banana: ", banana_f)

'''
print("True Battery: ", battery_t)
print("False Battery: ", battery_f)

print("True computer: ", computer_t)
print("False computer: ", computer_f)

print("True glass bottle: ", glass_bottle_t)
print("False glass bottle: ", glass_bottle_f)

print("True light blub: ", light_blub_t)
print("False light blub: ", light_blub_f)

print("True paper: ", paper_t)
print("False paper: ", paper_f)

print("True phone: ", phone_t)
print("False phone: ", phone_f)

print("True plastic: ", plastic_t)
print("False plastic: ", plastic_f)
'''


Label: Banana
Labels: battery
True Banana:  0
False Banana:  1


'\nprint("True Battery: ", battery_t)\nprint("False Battery: ", battery_f)\n\nprint("True computer: ", computer_t)\nprint("False computer: ", computer_f)\n\nprint("True glass bottle: ", glass_bottle_t)\nprint("False glass bottle: ", glass_bottle_f)\n\nprint("True light blub: ", light_blub_t)\nprint("False light blub: ", light_blub_f)\n\nprint("True paper: ", paper_t)\nprint("False paper: ", paper_f)\n\nprint("True phone: ", phone_t)\nprint("False phone: ", phone_f)\n\nprint("True plastic: ", plastic_t)\nprint("False plastic: ", plastic_f)\n'